In [3]:
import polars as pl

In [4]:
df = pl.scan_csv('/kaggle/input/data-dc/mimic-iv-2.1/hosp/labevents.csv')
df.schema

<ipython-input-4-f4a9715475ca>:2: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  df.schema


Schema([('labevent_id', Int64),
        ('subject_id', Int64),
        ('hadm_id', String),
        ('specimen_id', Int64),
        ('itemid', Int64),
        ('charttime', String),
        ('storetime', String),
        ('value', String),
        ('valuenum', Float64),
        ('valueuom', String),
        ('ref_range_lower', Float64),
        ('ref_range_upper', Float64),
        ('flag', String),
        ('priority', String),
        ('comments', String)])

In [5]:
import polars as pl

# Get total row count efficiently
total_rows = df.select(pl.count()).collect().item()

# Calculate 8% of total rows (rounded to integer)
n = int(total_rows * 0.08)

# Keep first n rows using lazy execution
sampled_df = (
    df
    .slice(0, n)  # Equivalent to `.head(n)`
    .collect()     # Execute the query
)

# Export to CSV (replace with your output path)
sampled_df.write_csv("first_8_percent.csv")

<ipython-input-5-a837e658789d>:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  total_rows = df.select(pl.count()).collect().item()


In [11]:
sampled_df = (
    df
    .slice(0, 2500000)  # Equivalent to `.head(n)`
    .collect()     # Execute the query
)

# Export to CSV (replace with your output path)
sampled_df.write_csv("2_5mil.csv")

In [6]:
import pandas as pd

In [1]:
import pandas as pd
df = pd.read_csv('/kaggle/input/reduced-df/2_5mil.csv')

In [2]:
df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,labevent_id,subject_id,hadm_id,specimen_id,itemid,charttime,storetime,value,valuenum,valueuom,ref_range_lower,ref_range_upper,flag,priority,comments
0,1,10000032,NaN,45421181,51237,2180-03-23 11:51:00,2180-03-23 15:15:00,1.4,1.4,NaN,0.9,1.1,abnormal,ROUTINE,NaN
1,2,10000032,NaN,45421181,51274,2180-03-23 11:51:00,2180-03-23 15:15:00,___,15.1,sec,9.4,12.5,abnormal,ROUTINE,VERIFIED.
2,3,10000032,NaN,52958335,50853,2180-03-23 11:51:00,2180-03-25 11:06:00,___,15.0,ng/mL,30.0,60.0,abnormal,ROUTINE,NEW ASSAY IN USE ___: DETECTS D2 AND D3 25-OH ...
3,4,10000032,NaN,52958335,50861,2180-03-23 11:51:00,2180-03-23 16:40:00,102,102.0,IU/L,0.0,40.0,abnormal,ROUTINE,NaN
4,5,10000032,NaN,52958335,50862,2180-03-23 11:51:00,2180-03-23 16:40:00,3.3,3.3,g/dL,3.5,5.2,abnormal,ROUTINE,NaN


In [3]:
df = df[['subject_id', 'hadm_id', 'itemid', 'valuenum']]
df.head

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


<bound method NDFrame.head of          subject_id     hadm_id  itemid  valuenum
0          10000032         NaN   51237      1.40
1          10000032         NaN   51274     15.10
2          10000032         NaN   50853     15.00
3          10000032         NaN   50861    102.00
4          10000032         NaN   50862      3.30
...             ...         ...     ...       ...
2499995    10220107  23406923.0   51265    176.00
2499996    10220107  23406923.0   51277     17.50
2499997    10220107  23406923.0   51279      3.12
2499998    10220107  23406923.0   51301      5.20
2499999    10220107  23406923.0   50868     14.00

[2500000 rows x 4 columns]>

In [10]:
pivot_df = df.pivot_table(
    index=['subject_id', 'hadm_id'],  # Group by these identifiers
    columns='itemid',                 # Convert unique itemids into columns
    values='valuenum',                # Populate columns with valuenum
    aggfunc='first'                   # Handle duplicates (use 'first', 'mean', etc.)
).reset_index()                       # Flatten the index

pivot_df.columns.name = None

pivot_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,subject_id,hadm_id,50801,50802,50803,50804,50805,50806,50808,50809,...,52276,52278,52281,52285,52286,52312,52358,52369,52391,52769
0,10000032,22595853.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000032,22841357.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10000032,25742920.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000032,29079034.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000084,23052089.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7270,10220107,23406923.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7271,10220107,25349952.0,NaN,4.0,NaN,28.0,NaN,106.0,1.12,110.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7272,10220107,27122498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
na = pivot_df.isnull().sum()

d = dict(na)

for i in d:
    if (d[i]/7274) > 0.5:
        pivot_df.drop(i, axis=1, inplace=True)

pivot_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,subject_id,hadm_id,50868,50882,50893,50902,50912,50931,50960,50970,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,22595853.0,9.0,28.0,7.8,105.0,0.3,99.0,1.7,3.6,...,1.5,33.4,33.8,99.0,71.0,16.6,32.3,15.3,3.80,4.2
1,10000032,22841357.0,14.0,25.0,7.8,92.0,0.3,71.0,1.9,3.3,...,1.5,34.5,35.0,99.0,137.0,16.3,NaN,15.7,3.60,6.6
2,10000032,25742920.0,11.0,26.0,8.6,95.0,0.6,95.0,2.4,3.9,...,1.5,35.9,34.8,103.0,133.0,15.8,32.5,16.0,3.36,7.5
3,10000032,29079034.0,14.0,21.0,9.3,102.0,0.5,115.0,2.3,2.4,...,1.8,34.9,34.1,102.0,94.0,19.5,34.2,15.8,3.40,4.1
4,10000084,23052089.0,15.0,19.0,9.9,98.0,0.8,100.0,2.0,3.7,...,1.0,31.8,33.5,95.0,263.0,11.1,23.4,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,12.0,27.0,8.8,104.0,1.2,142.0,2.1,2.8,...,3.3,29.6,32.3,92.0,185.0,34.9,57.1,14.8,3.65,3.5
7270,10220107,23406923.0,17.0,24.0,9.0,105.0,2.1,163.0,2.3,4.4,...,2.3,29.9,32.9,91.0,183.0,24.9,41.3,18.0,3.37,4.6
7271,10220107,25349952.0,15.0,25.0,8.7,104.0,1.2,104.0,2.0,3.5,...,1.0,29.1,34.5,85.0,258.0,10.8,70.8,13.5,4.27,6.4
7272,10220107,27122498.0,17.0,22.0,8.6,106.0,1.1,140.0,1.9,2.9,...,1.1,29.8,32.4,92.0,228.0,11.6,33.2,15.5,2.82,5.2


In [12]:
pivot_df.fillna(pivot_df.mean(numeric_only=True), inplace=True)

pivot_df

,subject_id,hadm_id,50868,50882,50893,50902,50912,50931,50960,50970,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,22595853.0,9.0,28.0,7.8,105.0,0.3,99.0,1.7,3.6,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,22841357.0,14.0,25.0,7.8,92.0,0.3,71.0,1.9,3.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,25742920.0,11.0,26.0,8.6,95.0,0.6,95.0,2.4,3.9,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,29079034.0,14.0,21.0,9.3,102.0,0.5,115.0,2.3,2.4,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
4,10000084,23052089.0,15.0,19.0,9.9,98.0,0.8,100.0,2.0,3.7,...,1.0,31.8,33.5,95.0,263.0,11.1,23.400000,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7269,10220107,23023366.0,12.0,27.0,8.8,104.0,1.2,142.0,2.1,2.8,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
7270,10220107,23406923.0,17.0,24.0,9.0,105.0,2.1,163.0,2.3,4.4,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
7271,10220107,25349952.0,15.0,25.0,8.7,104.0,1.2,104.0,2.0,3.5,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
7272,10220107,27122498.0,17.0,22.0,8.6,106.0,1.1,140.0,1.9,2.9,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [13]:
diag = pd.read_csv('/kaggle/input/patients-and-icdcode/diagnoses_icd.csv')
diag

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9
...,...,...,...,...,...
4752260,19999987,23865745,7,41401,9
4752261,19999987,23865745,8,78039,9
4752262,19999987,23865745,9,0413,9
4752263,19999987,23865745,10,36846,9


In [14]:
diag = diag[diag['seq_num'] == 1]
diag = diag[['hadm_id', 'icd_code', 'icd_version']]
diag

,hadm_id,icd_code,icd_version
0,22595853,5723,9
8,22841357,07071,9
16,25742920,07054,9
26,29079034,45829,9
39,25022803,30500,9
...,...,...,...
4752194,25744818,T8141XA,10
4752213,29734428,T8131XA,10
4752235,21033226,3453,9
4752247,26071774,43491,9


In [15]:
admissions = pd.read_csv('/kaggle/input/patients-and-icdcode/admissions.csv')
admissions = admissions[['subject_id', 'hadm_id']]
admissions

,subject_id,hadm_id
0,10000032,22595853
1,10000032,22841357
2,10000032,25742920
3,10000032,29079034
4,10000068,25022803
...,...,...
431083,19999828,25744818
431084,19999828,29734428
431085,19999840,21033226
431086,19999840,26071774


In [16]:
gender = pd.read_csv('/kaggle/input/patients-gender/patients.csv')
gender = gender[['subject_id', 'gender']]
gender

,subject_id,gender
0,10000032,F
1,10000048,F
2,10000068,F
3,10000084,M
4,10000102,F
...,...,...
299772,19999828,F
299773,19999829,F
299774,19999840,M
299775,19999914,F


In [17]:
final_df = pd.merge(
    gender, 
    admissions, 
    on='subject_id',  
    how='left'        
)

final_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,subject_id,gender,hadm_id
0,10000032,F,22595853.0
1,10000032,F,22841357.0
2,10000032,F,25742920.0
3,10000032,F,29079034.0
4,10000048,F,NaN
...,...,...,...
550113,19999829,F,NaN
550114,19999840,M,21033226.0
550115,19999840,M,26071774.0
550116,19999914,F,NaN


In [18]:
diag

,hadm_id,icd_code,icd_version
0,22595853,5723,9
8,22841357,07071,9
16,25742920,07054,9
26,29079034,45829,9
39,25022803,30500,9
...,...,...,...
4752194,25744818,T8141XA,10
4752213,29734428,T8131XA,10
4752235,21033226,3453,9
4752247,26071774,43491,9


In [19]:
final_df = pd.merge(
    final_df, 
    diag, 
    on='hadm_id', 
    how='left'     
)

final_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,subject_id,gender,hadm_id,icd_code,icd_version
0,10000032,F,22595853.0,5723,9.0
1,10000032,F,22841357.0,07071,9.0
2,10000032,F,25742920.0,07054,9.0
3,10000032,F,29079034.0,45829,9.0
4,10000048,F,NaN,NaN,NaN
...,...,...,...,...,...
550122,19999829,F,NaN,NaN,NaN
550123,19999840,M,21033226.0,3453,9.0
550124,19999840,M,26071774.0,43491,9.0
550125,19999914,F,NaN,NaN,NaN


In [20]:
pivot_df.drop('subject_id', axis=1, inplace=True)

In [21]:
final_df = pd.merge(
    final_df, 
    pivot_df, 
    on='hadm_id',  
    how='left'     
)

final_df

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,F,22595853.0,5723,9.0,9.0,28.0,7.8,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,F,22841357.0,07071,9.0,14.0,25.0,7.8,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,F,25742920.0,07054,9.0,11.0,26.0,8.6,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,F,29079034.0,45829,9.0,14.0,21.0,9.3,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
4,10000048,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550122,19999829,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550123,19999840,M,21033226.0,3453,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550124,19999840,M,26071774.0,43491,9.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
550125,19999914,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
final_df = final_df.dropna()

In [28]:
final_df.to_csv('combined_cleaned_df_final.csv')

In [27]:
final_df

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,F,22595853.0,5723,9.0,9.0,28.0,7.8,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,F,22841357.0,07071,9.0,14.0,25.0,7.8,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,F,25742920.0,07054,9.0,11.0,26.0,8.6,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,F,29079034.0,45829,9.0,14.0,21.0,9.3,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
6,10000084,M,23052089.0,G3183,10.0,15.0,19.0,9.9,98.0,0.8,...,1.0,31.8,33.5,95.0,263.0,11.1,23.400000,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,10220107,M,23023366.0,5779,9.0,12.0,27.0,8.8,104.0,1.2,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
11642,10220107,M,23406923.0,42823,9.0,17.0,24.0,9.0,105.0,2.1,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
11644,10220107,M,25349952.0,41401,9.0,15.0,25.0,8.7,104.0,1.2,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
11645,10220107,M,27122498.0,99811,9.0,17.0,22.0,8.6,106.0,1.1,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [29]:
from sklearn.preprocessing import LabelEncoder

gender_encoder = LabelEncoder()

final_df['gender'] = gender_encoder.fit_transform(final_df['gender'])

<ipython-input-29-680e827ac42a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['gender'] = gender_encoder.fit_transform(final_df['gender'])


In [31]:
final_df[['icd_code', 'gender']]

,icd_code,gender
0,5723,0
1,07071,0
2,07054,0
3,45829,0
6,G3183,1
...,...,...
11641,5779,1
11642,42823,1
11644,41401,1
11645,99811,1


In [32]:
final_df

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,0,22595853.0,5723,9.0,9.0,28.0,7.8,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,0,22841357.0,07071,9.0,14.0,25.0,7.8,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,0,25742920.0,07054,9.0,11.0,26.0,8.6,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,0,29079034.0,45829,9.0,14.0,21.0,9.3,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
6,10000084,1,23052089.0,G3183,10.0,15.0,19.0,9.9,98.0,0.8,...,1.0,31.8,33.5,95.0,263.0,11.1,23.400000,13.0,4.03,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,10220107,1,23023366.0,5779,9.0,12.0,27.0,8.8,104.0,1.2,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
11642,10220107,1,23406923.0,42823,9.0,17.0,24.0,9.0,105.0,2.1,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
11644,10220107,1,25349952.0,41401,9.0,15.0,25.0,8.7,104.0,1.2,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
11645,10220107,1,27122498.0,99811,9.0,17.0,22.0,8.6,106.0,1.1,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [38]:
df_icd9 = final_df[final_df['icd_version'] == 9]

In [39]:
df_icd10 = final_df[final_df['icd_version'] == 10]

In [40]:
df_icd9

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,10000032,0,22595853.0,5723,9.0,9.0,28.0,7.800000,105.0,0.3,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,10000032,0,22841357.0,07071,9.0,14.0,25.0,7.800000,92.0,0.3,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,10000032,0,25742920.0,07054,9.0,11.0,26.0,8.600000,95.0,0.6,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,10000032,0,29079034.0,45829,9.0,14.0,21.0,9.300000,102.0,0.5,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
14,10000248,1,20600184.0,9222,9.0,14.0,28.0,8.746584,101.0,0.8,...,1.0,29.6,35.6,83.0,286.0,10.8,35.200000,14.7,3.58,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,10220107,1,23023366.0,5779,9.0,12.0,27.0,8.800000,104.0,1.2,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
11642,10220107,1,23406923.0,42823,9.0,17.0,24.0,9.000000,105.0,2.1,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
11644,10220107,1,25349952.0,41401,9.0,15.0,25.0,8.700000,104.0,1.2,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
11645,10220107,1,27122498.0,99811,9.0,17.0,22.0,8.600000,106.0,1.1,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [41]:
df_icd10

,subject_id,gender,hadm_id,icd_code,icd_version,50868,50882,50893,50902,50912,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
6,10000084,1,23052089.0,G3183,10.0,15.0,19.0,9.900000,98.0,0.8,...,1.000000,31.8,33.5,95.0,263.0,11.100000,23.400000,13.0,4.03,8.6
7,10000084,1,29888819.0,R4182,10.0,13.0,24.0,9.600000,104.0,0.8,...,1.419679,31.8,32.8,97.0,257.0,15.545292,36.346258,13.1,4.09,8.7
11,10000117,0,22927623.0,R1310,10.0,20.0,22.0,8.746584,104.0,0.9,...,1.100000,30.9,33.6,92.0,248.0,11.600000,28.600000,12.1,4.82,5.0
12,10000117,0,27988844.0,S72012A,10.0,21.0,22.0,8.746584,103.0,0.9,...,1.100000,31.0,32.8,95.0,218.0,12.200000,28.900000,12.6,4.77,9.0
44,10000980,0,20897796.0,I130,10.0,17.0,20.0,9.400000,105.0,2.4,...,1.100000,31.3,32.4,97.0,176.0,12.200000,25.900000,15.1,2.88,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11620,10219419,1,24340279.0,C782,10.0,16.0,26.0,8.100000,100.0,0.6,...,1.400000,26.1,31.0,84.0,224.0,15.100000,28.300000,16.7,3.30,7.2
11623,10219419,1,29105022.0,A419,10.0,20.0,23.0,7.700000,98.0,1.6,...,1.500000,24.8,31.1,80.0,168.0,16.200000,26.900000,16.4,2.94,4.8
11624,10219451,1,27816171.0,L03112,10.0,11.0,26.0,8.800000,104.0,1.0,...,1.200000,31.2,33.2,94.0,156.0,13.100000,27.600000,12.0,3.85,9.0
11626,10219493,0,28975893.0,I7771,10.0,15.0,26.0,8.900000,97.0,0.7,...,1.100000,30.8,34.9,88.0,210.0,11.600000,61.700000,12.3,4.87,8.3


In [42]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
icd_encoded = encoder.fit_transform(df_icd9[['icd_code']])
icd_encoded_df = pd.DataFrame(
    icd_encoded,
    columns=encoder.get_feature_names_out(['icd_code'])
)

icd_encoded_df

,icd_code_00845,icd_code_0088,icd_code_0090,icd_code_01300,icd_code_01896,icd_code_0340,icd_code_035,icd_code_0380,icd_code_03811,icd_code_03812,...,icd_code_V5811,icd_code_V5812,icd_code_V5883,icd_code_V600,icd_code_V618,icd_code_V6284,icd_code_V714,icd_code_V7189,icd_code_V7281,icd_code_V7651
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
X = df_icd9.drop(['hadm_id', 'icd_code', 'subject_id'], axis=1)
X

,gender,icd_version,50868,50882,50893,50902,50912,50931,50960,50970,...,51237,51248,51249,51250,51265,51274,51275,51277,51279,51301
0,0,9.0,9.0,28.0,7.800000,105.0,0.3,99.0,1.700000,3.60000,...,1.5,33.4,33.8,99.0,71.0,16.6,32.300000,15.3,3.80,4.2
1,0,9.0,14.0,25.0,7.800000,92.0,0.3,71.0,1.900000,3.30000,...,1.5,34.5,35.0,99.0,137.0,16.3,36.346258,15.7,3.60,6.6
2,0,9.0,11.0,26.0,8.600000,95.0,0.6,95.0,2.400000,3.90000,...,1.5,35.9,34.8,103.0,133.0,15.8,32.500000,16.0,3.36,7.5
3,0,9.0,14.0,21.0,9.300000,102.0,0.5,115.0,2.300000,2.40000,...,1.8,34.9,34.1,102.0,94.0,19.5,34.200000,15.8,3.40,4.1
14,1,9.0,14.0,28.0,8.746584,101.0,0.8,93.0,1.944314,3.54061,...,1.0,29.6,35.6,83.0,286.0,10.8,35.200000,14.7,3.58,6.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11641,1,9.0,12.0,27.0,8.800000,104.0,1.2,142.0,2.100000,2.80000,...,3.3,29.6,32.3,92.0,185.0,34.9,57.100000,14.8,3.65,3.5
11642,1,9.0,17.0,24.0,9.000000,105.0,2.1,163.0,2.300000,4.40000,...,2.3,29.9,32.9,91.0,183.0,24.9,41.300000,18.0,3.37,4.6
11644,1,9.0,15.0,25.0,8.700000,104.0,1.2,104.0,2.000000,3.50000,...,1.0,29.1,34.5,85.0,258.0,10.8,70.800000,13.5,4.27,6.4
11645,1,9.0,17.0,22.0,8.600000,106.0,1.1,140.0,1.900000,2.90000,...,1.1,29.8,32.4,92.0,228.0,11.6,33.200000,15.5,2.82,5.2


In [44]:
y = icd_encoded_df
y

,icd_code_00845,icd_code_0088,icd_code_0090,icd_code_01300,icd_code_01896,icd_code_0340,icd_code_035,icd_code_0380,icd_code_03811,icd_code_03812,...,icd_code_V5811,icd_code_V5812,icd_code_V5883,icd_code_V600,icd_code_V618,icd_code_V6284,icd_code_V714,icd_code_V7189,icd_code_V7281,icd_code_V7651
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
y_labels = y.idxmax(axis=1)

In [46]:
X.columns = X.columns.astype(str)

In [47]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X)

X_scaled

array([[-0.99281271,  0.        , -1.5842577 , ...,  0.31320025,
        -0.10101255, -0.61857567],
       [-0.99281271,  0.        ,  0.04039634, ...,  0.51295987,
        -0.39831272, -0.31963363],
       [-0.99281271,  0.        , -0.93439608, ...,  0.66277958,
        -0.75507293, -0.20753037],
       ...,
       [ 1.00723932,  0.        ,  0.36532715, ..., -0.58571802,
         0.59764286, -0.34454547],
       [ 1.00723932,  0.        ,  1.01518877, ...,  0.41308006,
        -1.5577834 , -0.49401649],
       [ 1.00723932,  0.        ,  1.34011958, ..., -0.58571802,
        -0.81453296, -0.23244221]])

In [48]:
y

,icd_code_00845,icd_code_0088,icd_code_0090,icd_code_01300,icd_code_01896,icd_code_0340,icd_code_035,icd_code_0380,icd_code_03811,icd_code_03812,...,icd_code_V5811,icd_code_V5812,icd_code_V5883,icd_code_V600,icd_code_V618,icd_code_V6284,icd_code_V714,icd_code_V7189,icd_code_V7281,icd_code_V7651
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
import numpy as np
y_class = np.argmax(y.values, axis=1)
y_class

array([ 634,   28,   26, ...,  363, 1167,  414])

In [50]:
import xgboost as xgb

model = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=len(y.columns),  
    eval_metric='mlogloss',    
    n_estimators=100,
    random_state=42
)

model.fit(X_scaled, y_class)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=1203, num_parallel_tree=None, ...)

In [51]:
y_pred = model.predict(X_scaled)

In [52]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_class, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.90


In [53]:
import lightgbm as lgb
from sklearn.metrics import f1_score

lgb_params = {
    'objective': 'multiclass',
    'num_class': len(y.columns),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

train_data = lgb.Dataset(X_scaled, label=y_class)

lgb_model = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=200,
    valid_sets=[train_data],
    callbacks=[lgb.log_evaluation(50)]
)

y_pred_lgb = lgb_model.predict(X_scaled)

[50]	training's multi_logloss: 32.0039
[100]	training's multi_logloss: 29.7307
[150]	training's multi_logloss: 26.0748
[200]	training's multi_logloss: 24.4335


In [54]:
y_pred_lgb_class = np.argmax(y_pred_lgb, axis=1)
print("LightGBM F1-Score:", f1_score(y_class, y_pred_lgb_class, average='weighted'))

LightGBM F1-Score: 0.2286914579444655


In [55]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_class, y_pred_lgb_class)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.27


In [59]:
X_scaled.shape

(4575, 25)

In [60]:
y

,icd_code_00845,icd_code_0088,icd_code_0090,icd_code_01300,icd_code_01896,icd_code_0340,icd_code_035,icd_code_0380,icd_code_03811,icd_code_03812,...,icd_code_V5811,icd_code_V5812,icd_code_V5883,icd_code_V600,icd_code_V618,icd_code_V6284,icd_code_V714,icd_code_V7189,icd_code_V7281,icd_code_V7651
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4572,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4573,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# ------------------------------------------------------------------------------
# 1. Convert your data into PyTorch tensors
# ------------------------------------------------------------------------------
# Assume X and y are NumPy arrays:
#   X has shape (4575, 25)
#   y has shape (4575, 1203), where each row is one-hot.

X_tensor = torch.from_numpy(X_scaled).float()
y_tensor = torch.from_numpy(y_class).float()

# Convert one-hot vectors to integer class indices.
# If each row in y has exactly one "1" (i.e., single-label classification),
# the argmax along dimension 1 will yield the class index.
y_indices = y_tensor.long()

# ------------------------------------------------------------------------------
# 2. Create a Dataset and DataLoader
# ------------------------------------------------------------------------------
dataset = TensorDataset(X_tensor, y_indices)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ------------------------------------------------------------------------------
# 3. Define the network architecture
# ------------------------------------------------------------------------------
class SimpleNet(nn.Module):
    def __init__(self, input_size=25, hidden1=256, hidden2=128, num_classes=1203):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)  # No activation here; CrossEntropyLoss handles logits
        return x

model = SimpleNet(input_size=25, num_classes=1203)

# ------------------------------------------------------------------------------
# 4. Define the loss function and optimizer
# ------------------------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ------------------------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------------------------
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# ------------------------------------------------------------------------------
# 6. After training, evaluate or save your model as needed
# ------------------------------------------------------------------------------
print("Training complete.")

Epoch [1/50], Loss: 6.6803
Epoch [2/50], Loss: 5.9430
Epoch [3/50], Loss: 5.4299
Epoch [4/50], Loss: 4.9578
Epoch [5/50], Loss: 4.4835
Epoch [6/50], Loss: 4.0188
Epoch [7/50], Loss: 3.5697
Epoch [8/50], Loss: 3.1898
Epoch [9/50], Loss: 2.8777
Epoch [10/50], Loss: 2.6259
Epoch [11/50], Loss: 2.4033
Epoch [12/50], Loss: 2.1951
Epoch [13/50], Loss: 2.0029
Epoch [14/50], Loss: 1.8374
Epoch [15/50], Loss: 1.6829
Epoch [16/50], Loss: 1.5501
Epoch [17/50], Loss: 1.4144
Epoch [18/50], Loss: 1.3007
Epoch [19/50], Loss: 1.2025
Epoch [20/50], Loss: 1.1130
Epoch [21/50], Loss: 1.0250
Epoch [22/50], Loss: 0.9393
Epoch [23/50], Loss: 0.8749
Epoch [24/50], Loss: 0.8260
Epoch [25/50], Loss: 0.7513
Epoch [26/50], Loss: 0.7029
Epoch [27/50], Loss: 0.6596
Epoch [28/50], Loss: 0.6131
Epoch [29/50], Loss: 0.5758
Epoch [30/50], Loss: 0.5369
Epoch [31/50], Loss: 0.5126
Epoch [32/50], Loss: 0.4781
Epoch [33/50], Loss: 0.4530
Epoch [34/50], Loss: 0.4284
Epoch [35/50], Loss: 0.3998
Epoch [36/50], Loss: 0.3815
E

In [67]:
out = model(X_tensor)
out

tensor([[ -31.6345,  -29.7481,  -59.9273,  ...,  -76.7834,  -90.6945,
          -57.1826],
        [ -63.6343,  -65.3000,  -53.4917,  ..., -118.6806, -112.7767,
         -105.2140],
        [ -85.0768,  -78.4702,  -93.8643,  ..., -168.6218, -132.6541,
         -142.3732],
        ...,
        [ -38.4725,  -36.8378,  -45.5630,  ...,  -54.9622,  -53.3283,
          -49.3145],
        [ -51.4507,  -40.3320,  -92.6662,  ..., -109.0766, -116.0882,
          -64.1844],
        [ -56.5317,  -48.0490,  -82.6982,  ..., -153.7570, -109.4403,
          -76.5458]], grad_fn=<AddmmBackward0>)

In [68]:
nn_pred = out.argmax(dim=1).long()

In [69]:
nn_pred

tensor([ 634,   28,   26,  ...,  363, 1167,  414])

In [70]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_class, nn_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.95
